In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# ATTEMPT 1

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN'], axis=1, inplace=True)

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

# Determine which values to replace if counts are less than ...?
replace_app = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in replace_app:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [6]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
# Create a OneHotEncoder instancea
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

/Users/alexissimpson/opt/anaconda3/envs/data-b-18-unsupervisedlearning/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,NAME_13TH BOMB SQUADRON ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values
# Split the preprocessed data into a training and testing dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
#scaler = StandardScaler()

# Fit the StandardScaler
#X_scaler = scaler.fit(X_train)

# Scale the data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

/var/folders/1m/jpy67j3j70v3m3q2_99t0ryr0000gn/T/ipykernel_1767/2035127289.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL'],1).values


In [9]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)



In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

2022-07-30 16:19:16.744385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1568960   
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,571,421
Trainable params: 1,571,421
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 25s 28ms/step - loss: 0.4978 - accuracy: 0.7587
Epoch 2/100
804/804 [==============================] - 16s 20ms/step - loss: 0.1321 - accuracy: 0.9525
Epoch 3/100
804/804 [==============================] - 17s 21ms/step - loss: 0.0986 - accuracy: 0.9612
Epoch 4/100
804/804 [==============================] - 14s 18ms/step - loss: 0.0942 - accuracy: 0.9629
Epoch 5/100
804/804 [==============================] - ETA: 0s - loss: 0.0933 - accuracy: 0.9616
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 15s 18ms/step - loss: 0.0933 - accuracy: 0.9616
Epoch 6/100
804/804 [==============================] - 15s 18ms/step - loss: 0.0923 - accuracy: 0.9619
Epoch 7/100
804/804 [==============================] - 15s 19ms/step - loss: 0.0915 - accuracy: 0.9628
Epoch 8/100
804/804 [==============================] - 13s 16ms/step - loss: 0.0913 - accuracy: 0.9629
Epoch 9/100
804/804 [=================

804/804 [==============================] - 11s 13ms/step - loss: 0.0764 - accuracy: 0.9690
Epoch 64/100
804/804 [==============================] - 11s 14ms/step - loss: 0.0761 - accuracy: 0.9689
Epoch 65/100
801/804 [============================>.] - ETA: 0s - loss: 0.0765 - accuracy: 0.9691
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 13s 16ms/step - loss: 0.0764 - accuracy: 0.9691
Epoch 66/100
804/804 [==============================] - 14s 18ms/step - loss: 0.0760 - accuracy: 0.9691
Epoch 67/100
804/804 [==============================] - 14s 17ms/step - loss: 0.0762 - accuracy: 0.9691
Epoch 68/100
804/804 [==============================] - 12s 15ms/step - loss: 0.0759 - accuracy: 0.9688
Epoch 69/100
804/804 [==============================] - 12s 15ms/step - loss: 0.0758 - accuracy: 0.9694
Epoch 70/100
803/804 [============================>.] - ETA: 0s - loss: 0.0762 - accuracy: 0.9688
Epoch 70: saving model to checkpoints/weights.70

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 4s - loss: 0.8355 - accuracy: 0.6685 - 4s/epoch - 15ms/step
Loss: 0.8355117440223694, Accuracy: 0.6684548258781433


In [17]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")

In [18]:
# ATTEMPT 2

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               1961200   
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 30)                1530      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1,967,811
Trainable params: 1,967,811
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 20s 23ms/step - loss: 0.4731 - accuracy: 0.7766
Epoch 2/100
804/804 [==============================] - 25s 31ms/step - loss: 0.1421 - accuracy: 0.9481
Epoch 3/100
804/804 [==============================] - 23s 29ms/step - loss: 0.0967 - accuracy: 0.9610
Epoch 4/100
804/804 [==============================] - 20s 25ms/step - loss: 0.0928 - accuracy: 0.9615
Epoch 5/100
803/804 [============================>.] - ETA: 0s - loss: 0.0922 - accuracy: 0.9623
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 19s 24ms/step - loss: 0.0921 - accuracy: 0.9623
Epoch 6/100
804/804 [==============================] - 21s 26ms/step - loss: 0.0915 - accuracy: 0.9624
Epoch 7/100
804/804 [==============================] - 18s 22ms/step - loss: 0.0907 - accuracy: 0.9630
Epoch 8/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0897 - accuracy: 0.9628
Epoch 9/100
804/804 [=================

804/804 [==============================] - 15s 19ms/step - loss: 0.0759 - accuracy: 0.9688
Epoch 64/100
804/804 [==============================] - 16s 19ms/step - loss: 0.0757 - accuracy: 0.9692
Epoch 65/100
804/804 [==============================] - ETA: 0s - loss: 0.0757 - accuracy: 0.9691
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 16s 20ms/step - loss: 0.0757 - accuracy: 0.9691
Epoch 66/100
804/804 [==============================] - 17s 21ms/step - loss: 0.0755 - accuracy: 0.9693
Epoch 67/100
804/804 [==============================] - 17s 21ms/step - loss: 0.0760 - accuracy: 0.9686
Epoch 68/100
804/804 [==============================] - 17s 21ms/step - loss: 0.0755 - accuracy: 0.9691
Epoch 69/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0755 - accuracy: 0.9693
Epoch 70/100
802/804 [============================>.] - ETA: 0s - loss: 0.0756 - accuracy: 0.9693
Epoch 70: saving model to checkpoints/weights.70

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 5s - loss: 0.7393 - accuracy: 0.6616 - 5s/epoch - 18ms/step
Loss: 0.7392811179161072, Accuracy: 0.6615743637084961


In [25]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")

In [26]:
# ATTEMPT 3

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               1961200   
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 30)                1530      
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 1,967,811
Trainable params: 1,967,811
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [31]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 21s 24ms/step - loss: 0.4961 - accuracy: 0.7668
Epoch 2/100
804/804 [==============================] - 20s 25ms/step - loss: 0.1500 - accuracy: 0.9535
Epoch 3/100
804/804 [==============================] - 19s 24ms/step - loss: 0.1082 - accuracy: 0.9610
Epoch 4/100
804/804 [==============================] - 20s 25ms/step - loss: 0.0981 - accuracy: 0.9626
Epoch 5/100
803/804 [============================>.] - ETA: 0s - loss: 0.0933 - accuracy: 0.9624
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 20s 25ms/step - loss: 0.0933 - accuracy: 0.9624
Epoch 6/100
804/804 [==============================] - 24s 29ms/step - loss: 0.0907 - accuracy: 0.9628
Epoch 7/100
804/804 [==============================] - 21s 26ms/step - loss: 0.0891 - accuracy: 0.9642
Epoch 8/100
804/804 [==============================] - 18s 22ms/step - loss: 0.0879 - accuracy: 0.9636
Epoch 9/100
804/804 [=================

804/804 [==============================] - 16s 20ms/step - loss: 0.0760 - accuracy: 0.9689
Epoch 64/100
804/804 [==============================] - 15s 18ms/step - loss: 0.0759 - accuracy: 0.9688
Epoch 65/100
802/804 [============================>.] - ETA: 0s - loss: 0.0758 - accuracy: 0.9690
Epoch 65: saving model to checkpoints/weights.65.hdf5
804/804 [==============================] - 15s 18ms/step - loss: 0.0758 - accuracy: 0.9689
Epoch 66/100
804/804 [==============================] - 15s 19ms/step - loss: 0.0759 - accuracy: 0.9688
Epoch 67/100
804/804 [==============================] - 16s 20ms/step - loss: 0.0755 - accuracy: 0.9690
Epoch 68/100
804/804 [==============================] - 15s 19ms/step - loss: 0.0761 - accuracy: 0.9692
Epoch 69/100
804/804 [==============================] - 15s 19ms/step - loss: 0.0764 - accuracy: 0.9684
Epoch 70/100
804/804 [==============================] - ETA: 0s - loss: 0.0767 - accuracy: 0.9685
Epoch 70: saving model to checkpoints/weights.70

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 4s - loss: 0.5850 - accuracy: 0.6199 - 4s/epoch - 13ms/step
Loss: 0.585037112236023, Accuracy: 0.6199417114257812


In [33]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimzation.h5")